# SMS Spam Filter with Naive Bayes
In this project, I will be building a spam filter for SMS messages. I will use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that have already been classified. The data set can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

## Explore Data

In [1]:
# Import necessary modules
import pandas as pd

# Read dataset
sms = pd.read_csv('Datasets/SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
sms.shape

(5572, 2)

In [2]:
sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Find percentage of spam and ham msgs
sms['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

86.6% of SMS messages are ham (non-spam) and the remaining 13.4% of SMS messages are spam.

## Training & Test Set
To build the spam filter, I'll need to split the dataset into a training and test set to determine how good the model is at classifying *new* messages as either spam or ham. I will complete the split using an 80:20 split percentage, with 80% of the dataset being used for training, and 20% being usede for testing. This equates to:
* 4,458 messages for the training set
* 1,114 mesages for the test set

For this project, the goal is to create a spam filter that classifies new messages with an accuracy greater than 80%. 

In [4]:
# Randomise dataset
sms = sms.sample(frac=1, random_state=1)
# Split dataset
train = sms.copy().iloc[:4458]
test = sms.copy().iloc[4458:]
train.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)

# Percentage of spam and ham in both datasets
print(train['Label'].value_counts(normalize=True)*100)
print(test['Label'].value_counts(normalize=True)*100)

ham     86.54105
spam    13.45895
Name: Label, dtype: float64
ham     86.804309
spam    13.195691
Name: Label, dtype: float64


It can be seen that both the training and test data sets have similar spam percentages to the original full dataset.

## Letter Case & Punctuation
In order to use the Naive Bayes algorithm, the dataset needs to be cleaned and reformatted that will allow for easier extraction of the necessary information. To begin, I will remove the punctuation and make all words lower case.

In [5]:
# Make SMS msgs lowercase & remover punct.
train['SMS'] = train['SMS'].str.replace('\W', ' ', regex = True).str.lower()

## Creating the Vocabulary
The Naive Bayes algorithm will require a vocabulary count, which represents the number of unique words in the dataset. This will be created in the next cell.

In [6]:
# Transform msgs into lists
train['SMS'] = train['SMS'].str.split()

# Create vocabulary list
vocabulary = []
for msg in train['SMS']:
    for word in msg:
        vocabulary.append(word)
vocabulary = set(vocabulary) # Remove duplicates
vocabulary = list(vocabulary) # Transform back to list

## Final Training Set

To reformat the data into one that is easier to extract information from, I will use a dictionary that stores the number of times each word appears in the SMS message. The label column will remain intact, however the following columns would instead represent each unique word, with the values representing the number of times each message contained each unique word.

In [7]:
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary} # Instantiate count dictionary

# Populate dictionary with count values
for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)
train = pd.concat([train,word_counts_per_sms], axis = 1)
train.head()

,Label,SMS,youwanna,raed,sitter,ogunrinde,noline,hip,ard,eve,...,call,paru,23f,bloke,08719180219,tosend,swiss,itz,promotion,nobody
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants
The Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

$$P(\text{Spam} | w_1, w_2, \ldots, w_n) \propto P(\text{Spam}) \cdot \prod_{i=1}^{n} P(w_i | \text{Spam})$$
$$P(\text{Ham} | w_1, w_2, \ldots, w_n) \propto P(\text{Ham}) \cdot \prod_{i=1}^{n} P(w_i | \text{Ham})$$

Also, to calculate $P(w_i|Spam)$ and $P(w_i|Ham)$ inside the formulas above, I'll need to use the following equations:

$$P(w_i|\text{Spam}) = \frac{N_{w_i}|\text{Spam} + \alpha}{N_\text{Spam} + \alpha \cdot N_\text{Vocabulary}}$$

$$P(w_i|\text{Ham}) = \frac{N_{w_i}|\text{Ham} + \alpha}{N_\text{Ham} + \alpha \cdot N_\text{Vocabulary}}$$

Some of the terms in the four equations above will have the same value for every new message. Therefore, I can start by calculating:
* $P(Spam)$ and $P(Ham)$
* $N_\text{Spam}$, $N_\text{Ham}$, $N_\text{Vocabulary}$

When performing additive smoothing, I wil use Laplace smoothing and set $\alpha = 1$.

In [27]:
# Find P(Spam) & P(Ham)
p_spam = train['Label'].value_counts(normalize = True)['spam']
p_ham = train['Label'].value_counts(normalize = True)['ham']

# Find N spam, ham and vocab
spam = train[train['Label'] == 'spam']
n_spam = spam.iloc[:, 2:].sum().sum()

ham = train[train['Label'] == 'ham']
n_ham = ham.iloc[:, 2:].sum().sum()

# Find n vocab
n_vocabulary = len(vocabulary)

# Initiate alpha
alpha = 1

## Calculating Parameters
Since each word will have a different probability, I will have to calculate the probability of $P(w_i)|\text{Spam})$ and $P(w_i)|\text{Ham})$ for every word. Fortunately, as the data is all from the training set, the probabilities for each individual word is constant for every new message. By calculating each individual word's probability now, this will make the Naive Bayes algorithm much faster. When a new message comes in, most of the needed computations will have already been completed, enabling the algorithm to almost instantly classify the new message. If calculations are not performed beforehand, then the all calculations would have to be repeatedly performed for every new message, which is redundant due to it being an inefficient use of computer resources.

I will begin by creating the function for calculating each word's probability in spam or ham.

In [31]:
# Calculate word probability of spam or ham
def w_probs(word, is_spam):
    if is_spam:
        w_count = spam[word].sum()
        probability = (w_count + alpha) / (n_spam + (alpha * n_vocabulary))
        return probability
    else:
        w_count = ham[word].sum()
        probability = (w_count + alpha) / (n_ham + (alpha * n_vocabulary))
        return probability

I will now find the probability values for each word being spam and ham.

In [32]:
spam_dict = {}
ham_dict = {}
for w in vocabulary:
    spam_dict[w] = w_probs(w,True)
    ham_dict[w] = w_probs(w,False)

## Classifying A New Message
I can now use the constants and parameters calculated to create the spam filter.

In [42]:
# Naive Bayes spam filter function
import re
def spam_filter(message):
    # Format message appropriately
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    # Calculate probabilities
    spam_probs = 1
    ham_probs = 1
    for w in message:
        if w in vocabulary:
            spam_probs *= spam_dict[w]
            ham_probs *= ham_dict[w]
    
    spam_probs *= p_spam
    ham_probs *= p_ham
    # Output filter classification
    outcome = 'spam' if spam_probs > ham_probs else ('needs human classification' if spam_probs == ham_probs else 'ham')
    return outcome

Now, I will test the function with a spam and ham message.

In [43]:
spam_filter('WINNER!! This is the secret code to unlock the money: C3421.')

'spam'

In [44]:
spam_filter('"Sounds good, Tom, then see u there"')

'ham'

The function appears to be functioning as expected.

## Measuring the Spam Filter's Accuracy
I will now determine how well the spam filter does on the test set of 1,114 messages.

In [45]:
test['predicted'] = test['SMS'].apply(spam_filter)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now I can compare the predicted values with the actual values to measure how good the spam filter is with classifying new messages. To make the measurement, I'll use **accuracy** as a metric:
$$\text{Accuracy}=\frac{\text{number of correectly classified messages}}{\text{total number of classified messages}}$$

In [50]:
# Calculate accuracy metric
correct = 0
total = len(test)
for index, row in test.iterrows():
    if row['predicted'] == row['Label']:
        correct += 1
accuracy = correct/total
accuracy

0.9874326750448833

This spam filter has an accuracy of 98.74%, which is an excellent result considering the inital aim was accuracy greater than 80%. Further completion of this project would involve identifying the incorrectly predicted messages and assessing why the were incorrectly labeled and how this filter could be improved. Incorrectly labeled messages can be viewed below.

In [51]:
test[test['Label'] != test['predicted']]

,Label,SMS,predicted
114,spam,Not heard from U4 a while. Call me now am here...,ham
135,spam,More people are dogging in your area now. Call...,ham
152,ham,Unlimited texts. Limited minutes.,spam
159,ham,26th OF JULY,spam
284,ham,Nokia phone is lovly..,spam
293,ham,A Boy loved a gal. He propsd bt she didnt mind...,needs human classification
302,ham,No calls..messages..missed calls,spam
319,ham,We have sent JD for Customer Service cum Accou...,spam
504,spam,Oh my god! I've found your number again! I'm s...,ham
546,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham
